### 연습문제 
- Doc2Vec 라이브러리 이용한 감정에 분석 
- 데이터는 ratings_train.txt파일을 로드 
    - 특수 문자, 2칸 이상의 공백의 문자를 제거하는 문자열 좌우 공백 제거 정규화 함수 
    - document 컬럼의 데이터에서 중복 데이터를 제거 
    - 빈 텍스트, " "가 존재한다면 해당 행 데이터도 제거 
    - 상위의 5000개 정도 데이터를 이용
- 토큰화 함수 Komoran를 이용
    - 필요한 품사 : NNP, NNG, VV, VA, MAG, XR 만을 사용
    - 불용어 단어 : 하다, 되다, 이다, 것 , 수, 거 단어들은 제외
- 데이터에서 독립(document) , 종속(label) 변수로 데이터를 나눠주고 train, test 데이터셋을 나눠준다 비율은 8:2
- Doc2Vec 객체를 생성하여 학습 
    - 매개변수 
        - vector_size = 200
        - window = 5
        - min_count = 2
        - dm = 1
        - negative = 5
        - seed = 42
        - epochs = 50
    - 학습 시키는 데이터는 X_train
- X_train, X_test -> 문자열 데이터 -> infer_vector() 함수를 이용해서 임베딩  
- 고전 머신러닝 분류 모델을 이용하여 입베딩된 데이터를 독립 변수로 Y의 데이터들을 종속 변수로 학습으로 예측
    - 정확도를 확인 
    - LogisticRegression(max_iter = 2000, random_state = 42)
    - LinearSVC(random_state = 42)
    - 두개의 모델을 사용하여 정확도가 좋은 모델을 선택

In [ ]:
import pandas as pd
import re 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
# 라이브러리 설치 
# !pip install tqdm

In [ ]:
# 진행 상황들을 로그에 표시해주는 라이브러리 
from tqdm import tqdm

In [ ]:
from konlpy.tag import Komoran

In [ ]:
# 데이터 로드 
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.info()

In [ ]:
df2 = df.copy()

In [ ]:
# 150000개의 데이터중 결측치의 개수가 5개 확인 
# 결측치의 데이터가 매우 작기때문에 제거 
# 제거(drop) + 결측치(na) -> dropna()
# dropna() 함수를 Serise에서도 내장, DataFrame 내장
df2['document'] = df2['document'].dropna()
# df2['document'] 인덱스수 -> 150000
# df2['document'].dropna() 인덱스의 수 -> 149995

In [ ]:
df2.isna().sum()

In [ ]:
df2.loc[
    df2['document'].isna()
]

In [ ]:
# 실수가 많은 코드들
df2['document'] = df2['document'].dropna()
df2 = df2['document'].dropna()

In [ ]:
# 결측치를 제거 
df.dropna(inplace=True)

In [ ]:
# 텍스트 정규화 함수 
def nomalize(text):
    # text 매개변수에 들어오는 데이터?
        # df에 있는 document 컬럼의 values -> 리뷰 데이터
    # str(text) 사용하는 이유는?
        # 리뷰의 데이터가 문자가 아닌 경우 문자형으로 변경
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# df['documnet']에서 nomalize 함수를 이용
# nomalize에 들어가는 인자는 문자열이 기본
# nomalize(df['document']) 잘못된 부분
# apply() 함수 -> 데이터프레임에서 사용하는 함수 
# map() 함수를 비슷한 기능 각각의 원소들을 추출하여 어떤 작업(함수)들을 하는 기능
# df.head(3).apply(lambda x : str(x), axis=1)/
# df.head(3).map( lambda x : print(x) )
df = df.applymap(nomalize)

In [ ]:
# 빈 / 공백 텍스트 존재할수 있다.
df.loc[
    df['document'].isin( ["", " "] )
]

In [38]:
# 빈 텍스트 , 공백 텍스트  -> 길이가 1이하 
df = df.loc[
    df['document'].str.len() > 1
]

In [40]:
# document에서 중복된 데이터들은 제거 
df = df.drop_duplicates('document')

In [41]:
# 토큰화 함수를 생성 
# 특정 품사들만 선택 
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG', 'XR']
# 불용어 
stop_word = ['하다', '되다', '이다', '것', '수', '거']
# Komoran 객체를 생성 
komoran = Komoran()

def tokenize(text):
    tokens = []
    for word, pos in komoran.pos(text):
        # word : 단어
        # pos : 품사
        if pos in allow_pos and word not in stop_word:
            tokens.append(word)
    return tokens

In [43]:
df = df.head(5000)

In [ ]:
# token화 된 리스트를 데이터프레임에 새로운 컬럼에 추가 
tokenize_sentense = [ tokenize(val) for val in df['document'].values ]
tokenize_sentense

In [46]:
df['document_token'] = tokenize_sentense

In [47]:
df.head()

,id,document,label,document_token
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[더빙, 진짜, 짜증, 나, 목소리]"
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[포스터, 초딩, 영화, 오버, 연기, 가볍]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,[]
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[교도소, 이야기, 솔직히, 재미, 없, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[익살, 연기, 돋보이, 영화, 스파이더맨, 늙, 보이, 하, 커스틴 던스트, 너무나]"


In [ ]:
# 토큰화된 문서에 Tag를 부착
def tagged_docs(tokenize_data):
    tagged = []
    for d_id, toks in enumerate(tokenize_data):
        # toks의 길이가 0이라면 -> 학습에서 의미없는 문장
        if len(toks) == 0:
            continue
        tagged.append(
            TaggedDocument(
                words= toks, tags=[ f"DOC_{d_id}" ]
            )
        )
    return tagged

In [76]:
# 데이터프레임을 train,  test 셋으로 나눠준다. 
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state= 42, stratify=df['label']
)

In [77]:
train_df['label'].value_counts()

label
0    2007
1    1993
Name: count, dtype: int64

In [78]:
# Doc2Vec에서 사용할 데이터는 train_df의 document_token 컬럼의 데이터를 이용
tagged_train = tagged_docs(train_df['document_token'].values)

In [79]:
len(tagged_train)

3925

In [80]:
# Doc2Vec 객체를 생성하여 학습 ( 단어 사전 생성(build_vocab), 학습(train)  )
model = Doc2Vec(
    documents=tagged_train, 
    vector_size= 200, 
    window = 5, 
    dm = 1, 
    min_count = 2, 
    negative = 5, 
    seed = 42, 
    epochs= 50
)

In [81]:
# 모델 수동 학습
model2 = Doc2Vec(
    vector_size= 200, 
    window = 5, 
    dm = 1, 
    min_count = 2, 
    negative = 5, 
    seed = 42, 
    epochs= 50
)
# 단어 사전을 생성 
model2.build_vocab(tagged_train)
model2.train(tagged_train, total_examples=len(tagged_train), 
             epochs = 50)



In [82]:
# 단어 사전의 개수 확인  -> 단어별 임베딩 벡터 wv
print("단어 사전의 개수 : ", len(model.wv))
print('단어 사전의 개수 : ', len(model2.wv) )

단어 사전의 개수 :  2827
단어 사전의 개수 :  2827


In [85]:
len(model.dv)

3925

In [93]:
# 새로운 문장을 model에 infer_vector 함수를 이용하여 임베딩
def infer_vectors(model, norm_texts, epochs = 50):
    # norm_tests : 정규화 처리가 끝난 문서들
    # model : 임베딩 모델 
    vecs = []

    for text in norm_texts:
        # text : norm_tests의 각 원소들 대입 
        tokens = tokenize(text)
        # 토큰화된 데이터가 길이가 0인 경우 
        if len(tokens) == 0:
            # 비어있는 토큰 데이터는 영벡터 / 평균 벡터로 대체 가능 
            # 영벡터로 출력 
            # continue
            vecs.append( 
                np.zeros(model.vector_size, dtype=np.float32) )
        else:
            vec = model.infer_vector(tokens, epochs=epochs)
            vecs.append(vec)
    return np.vstack(vecs)

In [94]:
X_train = infer_vectors( model, train_df['document'].values )
Y_train = train_df['label'].values

X_test = infer_vectors(model, test_df['document'].values)
Y_test = test_df['label'].values

In [95]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

(4000, 200)
(4000,)
(1000, 200)
(1000,)


In [97]:
# 로지스틱회귀 모델과 선형 서포트벡터 분류 모델에 학습하고 
# 정확도 체크하는 함수 
def eval_clf(clf, X_tr, Y_tr, X_te, Y_te, model_name):
    # clf -> 분류 모델 입력값
    # tr -> train data
    # te -> test data
    # model_name -> 프린트에서 어떤 모델을 사용했는가?
    # 모델에 학습 
    clf.fit(X_tr, Y_tr)
    # 모델을 통한 예측
    pred = clf.predict(X_te)
    # 정확도 
    acc = accuracy_score(Y_te, pred)
    print(f"{model_name} 모델의 정확도는 :", round(acc, 4))
    # class report
    clf_report = classification_report(Y_te, pred)
    print(f"{model_name} 모델의 report : \n", clf_report)

In [98]:
# 모델들을 생성 
logi = LogisticRegression(max_iter=2000, random_state=42)
eval_clf(logi, X_train, Y_train, X_test, Y_test, "Logistic")

svc = LinearSVC(random_state=42)
eval_clf(svc, X_train, Y_train, X_test, Y_test, "LinearSVC")

Logistic 모델의 정확도는 : 0.742
Logistic 모델의 report : 
               precision    recall  f1-score   support

           0       0.74      0.75      0.75       502
           1       0.75      0.73      0.74       498

    accuracy                           0.74      1000
   macro avg       0.74      0.74      0.74      1000
weighted avg       0.74      0.74      0.74      1000

LinearSVC 모델의 정확도는 : 0.754
LinearSVC 모델의 report : 
               precision    recall  f1-score   support

           0       0.74      0.78      0.76       502
           1       0.77      0.73      0.75       498

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.75      0.75      0.75      1000

